Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [5]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [20]:
CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abakan,53.72,91.43
1,Achinsk,56.28,90.50
2,Almetyevsk,54.90,52.31
3,Angarsk,52.57,103.91
4,Arkhangelsk,64.57,40.53


In [17]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## Greedy + 2 opt 

In [21]:


# Initialization
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = [int(city)]

# Greedy approach for initial tour
while not np.all(visited):
    dist[:, city] = np.inf  # Mark current city's row as visited
    closest = np.argmin(dist[city])  # Find the nearest city
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city, closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))

# Close the loop back to the start
tsp.append(tsp[0])
logging.info(f"Initial path found with {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")



def two_opt_swap(route):
    best_route = route.copy()
    best_cost = tsp_cost(best_route)
    improvement = True

    while improvement:
        improvement = False
        for i in range(1, len(route) - 2):  # Avoid first and last elements
            for j in range(i + 1, len(route) - 1):
                if j - i == 1: continue  # Skip adjacent cities

                # Create a new route by swapping two edges
                new_route = route[:i] + route[i:j][::-1] + route[j:]
                new_cost = tsp_cost(new_route)
                
                # Update if we found a better route
                if new_cost < best_cost:
                    best_cost = new_cost
                    best_route = new_route
                    improvement = True
                    logging.debug(f"Improvement found with 2-opt: new length {best_cost:.2f}km")
        route = best_route

    return best_route

# Apply 2-opt to improve the initial greedy solution
optimized_tsp = two_opt_swap(tsp)
logging.info(f"Optimized path: Found a path of {len(optimized_tsp)-1} steps, total length {tsp_cost(optimized_tsp):.2f}km")


DEBUG:root:step: Abakan -> Krasnoyarsk (276.58km)
DEBUG:root:step: Krasnoyarsk -> Achinsk (161.71km)
DEBUG:root:step: Achinsk -> Kemerovo (296.59km)
DEBUG:root:step: Kemerovo -> Leninsk‐Kuznetskiy (74.76km)
DEBUG:root:step: Leninsk‐Kuznetskiy -> Prokopyevsk (91.87km)
DEBUG:root:step: Prokopyevsk -> Novokuznetsk (30.63km)
DEBUG:root:step: Novokuznetsk -> Biysk (187.38km)
DEBUG:root:step: Biysk -> Barnaul (132.82km)
DEBUG:root:step: Barnaul -> Novosibirsk (194.50km)
DEBUG:root:step: Novosibirsk -> Tomsk (206.90km)
DEBUG:root:step: Tomsk -> Seversk (14.97km)
DEBUG:root:step: Seversk -> Rubtsovsk (613.13km)
DEBUG:root:step: Rubtsovsk -> Omsk (647.47km)
DEBUG:root:step: Omsk -> Tobolsk (475.40km)


DEBUG:root:step: Tobolsk -> Tyumen (200.98km)
DEBUG:root:step: Tyumen -> Kurgan (189.69km)
DEBUG:root:step: Kurgan -> Kopeysk (236.87km)
DEBUG:root:step: Kopeysk -> Chelyabinsk (14.72km)
DEBUG:root:step: Chelyabinsk -> Miass (87.20km)
DEBUG:root:step: Miass -> Zlatoust (33.88km)
DEBUG:root:step: Zlatoust -> Pervouralsk (194.64km)
DEBUG:root:step: Pervouralsk -> Yekaterinburg (40.19km)
DEBUG:root:step: Yekaterinburg -> Kamensk‐Uralskiy (95.20km)
DEBUG:root:step: Kamensk‐Uralskiy -> Nizhniy Tagil (205.75km)
DEBUG:root:step: Nizhniy Tagil -> Perm (220.37km)
DEBUG:root:step: Perm -> Berezniki (159.92km)
DEBUG:root:step: Berezniki -> Izhevsk (353.17km)
DEBUG:root:step: Izhevsk -> Neftekamsk (107.05km)
DEBUG:root:step: Neftekamsk -> Naberezhnye Chelny (129.52km)
DEBUG:root:step: Naberezhnye Chelny -> Nizhnekamsk (31.95km)
DEBUG:root:step: Nizhnekamsk -> Almetyevsk (88.07km)
DEBUG:root:step: Almetyevsk -> Oktyabrskiy (88.27km)
DEBUG:root:step: Oktyabrskiy -> Ufa (170.17km)
DEBUG:root:step: Uf

## Greedy + evolutionary


In [19]:
# import numpy as np
import logging
import random

class GeneticTSP:
    def __init__(self, cities, dist_matrix, pop_size=100, generations=1000, mutation_rate=0.2):
        self.cities = cities
        self.dist_matrix = dist_matrix
        self.pop_size = pop_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.population = self.initialize_population()
    
    def initialize_population(self):
        # Generate initial population with greedy solution and random permutations
        population = [self.greedy_tour()]
        for _ in range(self.pop_size - 1):
            route = np.random.permutation(len(self.cities)).tolist()
            route.append(route[0])  # Make it a loop
            population.append(route)
        return population

    def greedy_tour(self):
        # Greedy approach for the initial tour
        visited = np.full(len(self.cities), False)
        dist = self.dist_matrix.copy()
        city = 0
        visited[city] = True
        tsp = [city]
        
        while not np.all(visited):
            dist[:, city] = np.inf  # Mark current city's row as visited
            closest = np.argmin(dist[city])
            visited[closest] = True
            city = closest
            tsp.append(city)
        
        tsp.append(tsp[0])  # Close the loop back to start
        return tsp
    
    def tsp_cost(self, route):
        # Calculate the total distance of the given route
        return sum(self.dist_matrix[route[i], route[i+1]] for i in range(len(route) - 1))
    
    def selection(self):
        # Sort population by fitness (shortest distance is better)
        ranked_population = sorted(self.population, key=lambda x: self.tsp_cost(x))
        # Select the top 50% of the population for crossover
        return ranked_population[:self.pop_size // 2]
    
    def crossover(self, parent1, parent2):
        # Ordered crossover (OX)
        size = len(parent1) - 1
        start, end = sorted(random.sample(range(size), 2))
        child = [-1] * size
        child[start:end] = parent1[start:end]

        for city in parent2:
            if city not in child:
                for i in range(size):
                    if child[i] == -1:
                        child[i] = city
                        break

        child.append(child[0])  # Close the loop
        return child
    
    def mutate(self, route):
        # Apply 2-opt mutation with a given probability
        if random.random() < self.mutation_rate:
            return self.two_opt_swap(route)
        return route
    
    def two_opt_swap(self, route):
        # Improved version of the 2-opt swap provided
        best_route = route.copy()
        best_cost = self.tsp_cost(best_route)
        improvement = True

        while improvement:
            improvement = False
            for i in range(1, len(route) - 2):
                for j in range(i + 1, len(route) - 1):
                    if j - i == 1:
                        continue
                    new_route = route[:i] + route[i:j][::-1] + route[j:]
                    new_cost = self.tsp_cost(new_route)
                    if new_cost < best_cost:
                        best_cost = new_cost
                        best_route = new_route
                        improvement = True
            route = best_route

        return best_route
    
    def evolve(self):
        for generation in range(self.generations):
            # Selection
            selected_population = self.selection()
            # Create next generation
            new_population = selected_population.copy()

            while len(new_population) < self.pop_size:
                parent1, parent2 = random.sample(selected_population, 2)
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                new_population.append(child)

            # Update the population with the new generation
            self.population = new_population

            # Track the best route and cost
            best_route = min(self.population, key=self.tsp_cost)
            best_cost = self.tsp_cost(best_route)
            logging.info(f"Generation {generation + 1}: Best route length {best_cost:.2f}km")

        # Final best route after all generations
        return min(self.population, key=self.tsp_cost)

# Example usage:
tsp_solver = GeneticTSP(CITIES, DIST_MATRIX, generations=20)
best_route = tsp_solver.evolve()
print("Best route:", best_route)
print("Total distance:", tsp_solver.tsp_cost(best_route))


INFO:root:Generation 1: Best route length 1345.54km
INFO:root:Generation 2: Best route length 1345.54km
INFO:root:Generation 3: Best route length 1345.54km
INFO:root:Generation 4: Best route length 1345.54km
INFO:root:Generation 5: Best route length 1345.54km
INFO:root:Generation 6: Best route length 1345.54km
INFO:root:Generation 7: Best route length 1345.54km
INFO:root:Generation 8: Best route length 1345.54km
INFO:root:Generation 9: Best route length 1345.54km
INFO:root:Generation 10: Best route length 1345.54km
INFO:root:Generation 11: Best route length 1345.54km


INFO:root:Generation 12: Best route length 1345.54km
INFO:root:Generation 13: Best route length 1345.54km
INFO:root:Generation 14: Best route length 1345.54km
INFO:root:Generation 15: Best route length 1345.54km
INFO:root:Generation 16: Best route length 1345.54km
INFO:root:Generation 17: Best route length 1345.54km
INFO:root:Generation 18: Best route length 1345.54km
INFO:root:Generation 19: Best route length 1345.54km
INFO:root:Generation 20: Best route length 1345.54km


Best route: [4, 3, 5, 6, 2, 0, 7, 1, 4]
Total distance: 1345.544956473311
